In [10]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker
import io
import yfinance as yf
import time

In [ ]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Used for Step 2 (Speed)
import yfinance as yf          # Used for Step 3 (Reliability)
import io
import json
import os
import time

# ==========================================
# CONFIGURATION
# ==========================================
MIN_PRICE = 5.00
MIN_VOLUME = 1_000_000       
MIN_CAP = 300_000_000        # $300M
MIN_CURRENT_RATIO = 1.2
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05              # 5% ROIC 
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']

CACHE_FILE = "financial_cache.json"
CACHE_EXPIRY_DAYS = 30 

# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache_data):
    try:
        with open(CACHE_FILE, 'w') as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    BS = Balance Sheet, FIN = Financials (Income Stmt)
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    # Get the most recent column (usually first)
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])
        total_revenue = get_val(fin, ['Total Revenue'])
        
        if total_assets == 0 or total_liab == 0: return 0

        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets
        
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

# ==========================================
# STEP 1: FETCH UNIVERSE
# ==========================================
def get_us_universe():
    print("--- STEP 1: Downloading Universe from NasdaqTrader.txt ---")
    url = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
    try:
        s = requests.get(url).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        df = df[(df['Test Issue'] == 'N') & (df['ETF'] == 'N')]
        symbol_list = df['Symbol'].astype(str).tolist()
        clean_list = [x.replace('$', '-') for x in symbol_list if len(x) < 5] 
        print(f" -> Found {len(clean_list)} potential candidates.")
        return clean_list
    except Exception as e:
        print(f"Error fetching universe: {e}")
        return []

# ==========================================
# STEP 2: THE LIGHTWEIGHT SIEVE (YahooQuery for Speed)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500 
    survivors = []
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        if i % 2 == 0: print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules('summaryProfile summaryDetail financialData price defaultKeyStatistics')
            
            for symbol, data in df_modules.items():
                if isinstance(data, str): continue 
                try:
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: price = 0
                    
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                         vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: cap = 0
                    
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)
                    op_margins = fin_data.get('operatingMargins', 0)
                    
                    if curr_ratio is None: curr_ratio = 0
                    if op_margins is None: op_margins = 0

                    # FILTERS
                    if price < MIN_PRICE: continue
                    if cap < MIN_CAP: continue
                    if vol < MIN_VOLUME: continue 
                    if any(x in sector for x in EXCLUDED_SECTORS): continue
                    if curr_ratio < MIN_CURRENT_RATIO: continue
                    if op_margins <= 0: continue 
                    
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)
                    })
                except: continue
        except: continue
            
    return pd.DataFrame(survivors)

# ==========================================
# STEP 3: THE DEEP DIVE (yfinance for Reliability)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    print("    (Switching to yfinance for reliability. This takes ~1 sec per stock.)")
    
    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400
    
    final_data = []
    
    for i, ticker in enumerate(tickers):
        # Progress Print
        if i % 10 == 0: print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: continue
            if cached_data['int_cov'] < MIN_INTEREST_COVERAGE: continue
            if cached_data['roic'] < MIN_ROIC: continue
            
            # Rebuild row
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            op_margin_pct = base_row['Op Margin %'] / 100
            tier = "Fortress" if op_margin_pct >= FORTRESS_MARGIN_THRESHOLD else "Strong"
            
            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'Sector': base_row['Sector'],
                'Z-Score': cached_data['z_score'],
                'ROIC %': round(cached_data['roic'] * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': cached_data['int_cov'],
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            continue

        # 2. FETCH NEW DATA (yfinance)
        try:
            stock = yf.Ticker(ticker)
            
            # Fetch Financials (DataFrame format)
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                cache[ticker] = {'timestamp': current_time, 'z_score': 0, 'roic': -999, 'int_cov': -999}
                continue
            
            # --- EXTRACT METRICS SAFELY ---
            # Helper to get first column (most recent year)
            def get_item(df, keys):
                for k in keys:
                    if k in df.index:
                        return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            # Calculations
            int_exp = abs(int_exp)
            if int_exp == 0: int_cov = 100
            else: int_cov = ebit / int_exp
            
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0: roic = 0
            else: roic = ebit / invested_cap
            
            # Z-Score
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            # UPDATE CACHE
            cache[ticker] = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            
            # FILTERS
            if int_cov < MIN_INTEREST_COVERAGE: continue
            if roic < MIN_ROIC: continue
            
            op_margin_pct = base_row['Op Margin %'] / 100
            tier = "Fortress" if op_margin_pct >= FORTRESS_MARGIN_THRESHOLD else "Strong"

            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            # print(f"Error on {ticker}: {e}")
            continue

    # Save cache at the end
    save_cache(cache)
    return pd.DataFrame(final_data)

# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    tickers = get_us_universe()
    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)
        if not survivors_df.empty:
            print(f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                final_results = final_results.sort_values(by=['Tier', 'Z-Score'], ascending=[True, False])
                print("\n" + "="*60)
                print("ALL-IN-ONE NON-FINANCIAL FILTER RESULTS")
                print("="*60)
                fortress_count = len(final_results[final_results['Tier'] == 'Fortress'])
                print(f"Total Matches: {len(final_results)} | Fortress: {fortress_count}")
                
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                print(final_results.head(50))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Downloading Universe from NasdaqTrader.txt ---
 -> Found 6014 potential candidates.

--- STEP 2: Running 'Lightweight' Filter on 6014 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 3/13...
 -> Processing Batch 5/13...
 -> Processing Batch 7/13...
 -> Processing Batch 9/13...
 -> Processing Batch 11/13...
 -> Processing Batch 13/13...

✅ Step 2 Complete. 241 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 241 Survivors ---
    (Switching to yfinance for reliability. This takes ~1 sec per stock.)
 -> Analyzing 1/241: AUPH...
 -> Analyzing 11/241: BAH...
 -> Analyzing 21/241: BILI...
 -> Analyzing 31/241: BSX...
 -> Analyzing 41/241: CAT...
 -> Analyzing 51/241: CELH...
 -> Analyzing 61/241: CLS...
 -> Analyzing 71/241: CPRT...
 -> Analyzing 81/241: CTSH...
 -> Analyzing 91/241: DAR...
 -> Analyzing 101/241: DINO...
 -> Analyzing 111/241: DUOL...
 -> Analyzing 121/241: ELV...
 -> Analyzing 131/241: ETSY...
 -> Analyzing 141/241: FFIV...
 ->

In [12]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Used for Step 2 (Speed)
import yfinance as yf          # Used for Step 3 (Reliability)
import io
import json
import os
import time

# ==========================================
# CONFIGURATION
# ==========================================
MIN_PRICE = 5.00
MIN_VOLUME = 1_000_000       
MIN_CAP = 300_000_000        # $300M
MIN_CURRENT_RATIO = 1.2
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05              # 5% ROIC 
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']

CACHE_FILE = "financial_cache.json"
CACHE_EXPIRY_DAYS = 30 

# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache_data):
    try:
        with open(CACHE_FILE, 'w') as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])
        total_revenue = get_val(fin, ['Total Revenue'])
        
        if total_assets == 0 or total_liab == 0: return 0

        A = (current_assets - current_liab) / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = total_revenue / total_assets
        
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

# ==========================================
# STEP 1: FETCH UNIVERSE
# ==========================================
def get_us_universe():
    print("--- STEP 1: Downloading Universe from NasdaqTrader.txt ---")
    url = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
    try:
        s = requests.get(url).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        df = df[(df['Test Issue'] == 'N') & (df['ETF'] == 'N')]
        symbol_list = df['Symbol'].astype(str).tolist()
        clean_list = [x.replace('$', '-') for x in symbol_list if len(x) < 5] 
        print(f" -> Found {len(clean_list)} potential candidates.")
        return clean_list
    except Exception as e:
        print(f"Error fetching universe: {e}")
        return []

# ==========================================
# STEP 2: THE LIGHTWEIGHT SIEVE (YahooQuery)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500 
    survivors = []
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        if i % 2 == 0: print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules('summaryProfile summaryDetail financialData price defaultKeyStatistics')
            
            for symbol, data in df_modules.items():
                if isinstance(data, str): continue 
                try:
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: price = 0
                    
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                         vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: cap = 0
                    
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)
                    op_margins = fin_data.get('operatingMargins', 0)
                    
                    if curr_ratio is None: curr_ratio = 0
                    if op_margins is None: op_margins = 0

                    if price < MIN_PRICE: continue
                    if cap < MIN_CAP: continue
                    if vol < MIN_VOLUME: continue 
                    if any(x in sector for x in EXCLUDED_SECTORS): continue
                    if curr_ratio < MIN_CURRENT_RATIO: continue
                    if op_margins <= 0: continue 
                    
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)
                    })
                except: continue
        except: continue
            
    return pd.DataFrame(survivors)

# ==========================================
# STEP 3: THE DEEP DIVE (yfinance)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    print("    (Switching to yfinance for reliability. This takes ~1 sec per stock.)")
    
    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400
    
    final_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 10 == 0: print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: continue
            if cached_data['int_cov'] < MIN_INTEREST_COVERAGE: continue
            if cached_data['roic'] < MIN_ROIC: continue
            
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            op_margin_pct = base_row['Op Margin %'] / 100
            tier = "Fortress" if op_margin_pct >= FORTRESS_MARGIN_THRESHOLD else "Strong"
            
            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'Sector': base_row['Sector'],
                'Z-Score': cached_data['z_score'],
                'ROIC %': round(cached_data['roic'] * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': cached_data['int_cov'],
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            continue

        # 2. FETCH NEW DATA (yfinance)
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                cache[ticker] = {'timestamp': current_time, 'z_score': 0, 'roic': -999, 'int_cov': -999}
                continue
            
            def get_item(df, keys):
                for k in keys:
                    if k in df.index:
                        return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            int_exp = abs(int_exp)
            if int_exp == 0: int_cov = 100
            else: int_cov = ebit / int_exp
            
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0: roic = 0
            else: roic = ebit / invested_cap
            
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            cache[ticker] = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            
            if int_cov < MIN_INTEREST_COVERAGE: continue
            if roic < MIN_ROIC: continue
            
            op_margin_pct = base_row['Op Margin %'] / 100
            tier = "Fortress" if op_margin_pct >= FORTRESS_MARGIN_THRESHOLD else "Strong"

            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue

    save_cache(cache)
    return pd.DataFrame(final_data)

# ==========================================
# MAIN EXECUTION (UPDATED)
# ==========================================
if __name__ == "__main__":
    tickers = get_us_universe()
    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)
        if not survivors_df.empty:
            print(f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                # 1. Sort Global Results
                final_results = final_results.sort_values(by=['Tier', 'Z-Score'], ascending=[True, False])
                
                # 2. CREATE VARIABLES FOR DATA WRANGLER
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                
                # 3. SAVE TO CSV (Optional: Open these in Data Wrangler)
                fortress_df.to_csv("fortress_stocks.csv", index=False)
                strong_df.to_csv("strong_stocks.csv", index=False)
                
                print("\n" + "="*60)
                print("RESULTS SEPARATED")
                print("="*60)
                print(f"Fortress DataFrame: {len(fortress_df)} stocks (Saved to 'fortress_stocks.csv')")
                print(f"Strong DataFrame:   {len(strong_df)} stocks (Saved to 'strong_stocks.csv')")
                
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                print("\n--- FORTRESS TOP 20 ---")
                print(fortress_df.head(20))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Downloading Universe from NasdaqTrader.txt ---
 -> Found 6014 potential candidates.

--- STEP 2: Running 'Lightweight' Filter on 6014 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 3/13...
 -> Processing Batch 5/13...
 -> Processing Batch 7/13...
 -> Processing Batch 9/13...
 -> Processing Batch 11/13...
 -> Processing Batch 13/13...

✅ Step 2 Complete. 256 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 256 Survivors ---
    (Switching to yfinance for reliability. This takes ~1 sec per stock.)
 -> Analyzing 1/256: A...
 -> Analyzing 11/256: ADM...
 -> Analyzing 21/256: ALAB...
 -> Analyzing 31/256: AMD...
 -> Analyzing 41/256: ANF...
 -> Analyzing 51/256: ARHS...
 -> Analyzing 61/256: ASML...
 -> Analyzing 71/256: AVTR...
 -> Analyzing 81/256: BG...
 -> Analyzing 91/256: BMY...
 -> Analyzing 101/256: BWA...
 -> Analyzing 111/256: CDE...
 -> Analyzing 121/256: CGNX...
 -> Analyzing 131/256: COHR...
 -> Analyzing 141/256: CRMD...
 -> Anal